# <center> Projet :  Fruits </center>



<img src="https://img.passeportsante.net/1200x675/2020-01-07/i93188-.webp" width=492>


### <center>Thème : Classification de Fruits</center>
### <center>https://openclassrooms.com/fr/paths/164/projects/633/assignment<center>


Votre start-up souhaite dans un premier temps se faire connaître en mettant à disposition du grand public une application mobile qui permettrait aux utilisateurs de prendre en photo un fruit et d'obtenir des informations sur ce fruit.

Pour la start-up, cette application permettrait de sensibiliser le grand public à la biodiversité des fruits et de mettre en place une première version du moteur de classification des images de fruits.

De plus, le développement de l’application mobile permettra de construire une première version de l'architecture Big Data nécessaire.

In [1]:
# Librairies utilises

#Data management/data viz
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Statistiques/ML
#import sklearn
#from sparkdl import DeepImagePredictor

#tensorflow
import tensorflow
print(tensorflow.__version__)
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array


#Keras
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model


#Image
#from PIL import Image
#from IPython.display import  display
#from PIL import ImageFilter
#from matplotlib.image import imread

#Spark
import pyspark
from pyspark import SparkContext
from pyspark.sql.types import Row, _create_row, _parse_datatype_json_string
from pyspark.sql import DataFrame, SparkSession
from pyspark.ml.image import ImageSchema
from pyspark.sql import SparkSession
from pyspark.ml.image import ImageSchema
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA
from pyspark.sql.functions import element_at, split, col, pandas_udf, PandasUDFType, udf
from pyspark.ml.feature import PCA
from pyspark.ml.feature import StandardScaler
from pyspark.ml.linalg import Vectors, VectorUDT, DenseVector

#Amazon
import boto3


#Autre
import pathlib
import time
import os
from io import StringIO
import io

#Afficher toute les colonnes
#pd.set_option('display.max_columns', None)

#pd.set_option('display.max_rows', None)

2.9.1


In [2]:
# Travailler en Local ou sur le cloud
LOCAL=0
# Repertoire
# Data_low est une version très réduite
# Data_medium est une version intermédiaire
# Data est la version complète
REPERTOIRE="Data_low/*"    #"Data_low/*","Data_medium/*", "Data/*"

In [3]:
try :
    sc = SparkContext()
    spark=SparkSession(sc)
except :
    print("Session spark déjà active")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/01 23:22:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Importation des données

Dans cette partie nous importons notre data.

In [4]:
#### Autre manière
#df = spark.read.format("image").option("dropInvalid", True).option("numFeatures", "780").load(REPERTOIRE)
#df = df.withColumn("Classe", element_at(split(df["image.origin"], "/"), -2))
#print("Nous avons" ,df.count(), "images de fruits.")
#df.select("Classe","image.origin").show(truncate=False) #, "image.width", "image.height"
#df.select("image.origin").first()
#df.printSchema()

In [4]:
df_bin= spark.read.format("binaryFile").option("pathGlobFilter", "*.jpg").option("recursiveFileLookup", "true").load(REPERTOIRE)
#df = df.withColumn("Classe", element_at(split(df["image.origin"], "/"), -2))
df_bin = df_bin.withColumn("classe", element_at(split(df_bin["path"], "/"), -2))
print("Nous avons" ,df_bin.count(), "images de fruits.")
df_bin.show()
#Ok

Nous avons 18 images de fruits.
+--------------------+-------------------+------+--------------------+---------+
|                path|   modificationTime|length|             content|   classe|
+--------------------+-------------------+------+--------------------+---------+
|file:/Users/samue...|2021-09-12 19:24:46| 55981|[FF D8 FF E0 00 1...|Concombre|
|file:/Users/samue...|2021-09-12 19:24:46| 50736|[FF D8 FF E0 00 1...|Concombre|
|file:/Users/samue...|2021-09-12 19:24:46| 50560|[FF D8 FF E0 00 1...|Concombre|
|file:/Users/samue...|2021-09-12 19:24:46| 50543|[FF D8 FF E0 00 1...|Concombre|
|file:/Users/samue...|2021-09-12 19:24:46| 50051|[FF D8 FF E0 00 1...|Concombre|
|file:/Users/samue...|2021-09-12 19:24:46| 50029|[FF D8 FF E0 00 1...|Concombre|
|file:/Users/samue...|2021-09-12 19:24:04| 35060|[FF D8 FF E0 00 1...|    Pomme|
|file:/Users/samue...|2021-09-12 19:24:04| 34794|[FF D8 FF E0 00 1...|    Pomme|
|file:/Users/samue...|2021-09-12 19:24:04| 34610|[FF D8 FF E0 00 1...|    Pom

In [5]:
df_bin.select("path").first()

Row(path='file:/Users/samueljacquot/Desktop/P8_SAMUEL_JACQUOT/Data_low/Concombre/r0_4.jpg')

In [6]:
df_bin.printSchema()

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- classe: string (nullable = true)



In [7]:
try :
    s3 = boto3.resource('s3', region_name='us-east-2')
    bucket = s3.Bucket('classification-fruits')
    #object = bucket.Object('tiles/10/S/DG/2015/12/7/0/B01.jp2')

    #file_stream = io.StringIO()
    #object.download_fileobj(file_stream)
        #img = mpimg.imread(file_stream)
    s3 = boto3.resource('s3',aws_access_key_id=ACCESS_ID,aws_secret_access_key= ACCESS_KEY)
    object = bucket.Object('Data_low/Carotte/r0_0.jpg')

    file_stream = io.StringIO()
    object.download_fileobj(file_stream)
    img = mpimg.imread(file_stream)
except :
    print("Amazon S3 non opérationnel")

Amazon S3 non opérationnel


# Extraction de features CNN

https://docs.databricks.com/_static/notebooks/deep-learning/deep-learning-transfer-learning-keras.html

In [8]:
df_bin.select("path", "classe").show()

+--------------------+---------+
|                path|   classe|
+--------------------+---------+
|file:/Users/samue...|Concombre|
|file:/Users/samue...|Concombre|
|file:/Users/samue...|Concombre|
|file:/Users/samue...|Concombre|
|file:/Users/samue...|Concombre|
|file:/Users/samue...|Concombre|
|file:/Users/samue...|    Pomme|
|file:/Users/samue...|    Pomme|
|file:/Users/samue...|    Pomme|
|file:/Users/samue...|    Pomme|
|file:/Users/samue...|    Pomme|
|file:/Users/samue...|  Carotte|
|file:/Users/samue...|  Carotte|
|file:/Users/samue...|  Carotte|
|file:/Users/samue...|  Carotte|
|file:/Users/samue...|  Carotte|
|file:/Users/samue...|  Carotte|
|file:/Users/samue...|    Pomme|
+--------------------+---------+



In [9]:
model = ResNet50(include_top=False)
model.summary()  

2022-09-01 23:22:44.465524: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        ['conv1_pad[0][0]']              
                                64)                                                        

                                                                                                  
 conv2_block2_add (Add)         (None, None, None,   0           ['conv2_block1_out[0][0]',       
                                256)                              'conv2_block2_3_bn[0][0]']      
                                                                                                  
 conv2_block2_out (Activation)  (None, None, None,   0           ['conv2_block2_add[0][0]']       
                                256)                                                              
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, None, None,   16448       ['conv2_block2_out[0][0]']       
                                64)                                                               
                                                                                                  
 conv2_blo

                                128)                                                              
                                                                                                  
 conv3_block2_2_bn (BatchNormal  (None, None, None,   512        ['conv3_block2_2_conv[0][0]']    
 ization)                       128)                                                              
                                                                                                  
 conv3_block2_2_relu (Activatio  (None, None, None,   0          ['conv3_block2_2_bn[0][0]']      
 n)                             128)                                                              
                                                                                                  
 conv3_block2_3_conv (Conv2D)   (None, None, None,   66048       ['conv3_block2_2_relu[0][0]']    
                                512)                                                              
          

 conv4_block1_1_bn (BatchNormal  (None, None, None,   1024       ['conv4_block1_1_conv[0][0]']    
 ization)                       256)                                                              
                                                                                                  
 conv4_block1_1_relu (Activatio  (None, None, None,   0          ['conv4_block1_1_bn[0][0]']      
 n)                             256)                                                              
                                                                                                  
 conv4_block1_2_conv (Conv2D)   (None, None, None,   590080      ['conv4_block1_1_relu[0][0]']    
                                256)                                                              
                                                                                                  
 conv4_block1_2_bn (BatchNormal  (None, None, None,   1024       ['conv4_block1_2_conv[0][0]']    
 ization) 

                                                                                                  
 conv4_block3_3_bn (BatchNormal  (None, None, None,   4096       ['conv4_block3_3_conv[0][0]']    
 ization)                       1024)                                                             
                                                                                                  
 conv4_block3_add (Add)         (None, None, None,   0           ['conv4_block2_out[0][0]',       
                                1024)                             'conv4_block3_3_bn[0][0]']      
                                                                                                  
 conv4_block3_out (Activation)  (None, None, None,   0           ['conv4_block3_add[0][0]']       
                                1024)                                                             
                                                                                                  
 conv4_blo

 ization)                       256)                                                              
                                                                                                  
 conv4_block6_2_relu (Activatio  (None, None, None,   0          ['conv4_block6_2_bn[0][0]']      
 n)                             256)                                                              
                                                                                                  
 conv4_block6_3_conv (Conv2D)   (None, None, None,   263168      ['conv4_block6_2_relu[0][0]']    
                                1024)                                                             
                                                                                                  
 conv4_block6_3_bn (BatchNormal  (None, None, None,   4096       ['conv4_block6_3_conv[0][0]']    
 ization)                       1024)                                                             
          

 conv5_block3_1_conv (Conv2D)   (None, None, None,   1049088     ['conv5_block2_out[0][0]']       
                                512)                                                              
                                                                                                  
 conv5_block3_1_bn (BatchNormal  (None, None, None,   2048       ['conv5_block3_1_conv[0][0]']    
 ization)                       512)                                                              
                                                                                                  
 conv5_block3_1_relu (Activatio  (None, None, None,   0          ['conv5_block3_1_bn[0][0]']      
 n)                             512)                                                              
                                                                                                  
 conv5_block3_2_conv (Conv2D)   (None, None, None,   2359808     ['conv5_block3_1_relu[0][0]']    
          

In [10]:
bc_model_weights = sc.broadcast(model.get_weights())

def model_fn():
  """
  Returns a ResNet50 model with top layer removed and broadcasted pretrained weights.
  """
  model = ResNet50(weights=None, include_top=False)
  model.set_weights(bc_model_weights.value)
  return model

In [11]:

def preprocess(content):
  """
  Preprocesses raw image bytes for prediction.
  """
  img = Image.open(io.BytesIO(content)).resize([224, 224])
  arr = img_to_array(img)
  return preprocess_input(arr)

def featurize_series(model, content_series):
  """
  Featurize a pd.Series of raw images using the input model.
  :return: a pd.Series of image features
  """
  input = np.stack(content_series.map(preprocess))
  preds = model.predict(input)
  # For some layers, output features will be multi-dimensional tensors.
  # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
  output = [p.flatten() for p in preds]
  return pd.Series(output)

In [12]:
@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
  '''
  This method is a Scalar Iterator pandas UDF wrapping our featurization function.
  The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).
  
  :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
  '''
  # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
  # for multiple data batches.  This amortizes the overhead of loading big models.
  model = model_fn()
  for content_series in content_series_iter:
    yield featurize_series(model, content_series)

/Users/samueljacquot/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/pandas/functions.py:394: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [13]:
# Pandas UDFs on large records (e.g., very large images) can run into Out Of Memory (OOM) errors.
# If you hit such errors in the cell below, try reducing the Arrow batch size via `maxRecordsPerBatch`.
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

In [14]:
df_bin.show()

+--------------------+-------------------+------+--------------------+---------+
|                path|   modificationTime|length|             content|   classe|
+--------------------+-------------------+------+--------------------+---------+
|file:/Users/samue...|2021-09-12 19:24:46| 55981|[FF D8 FF E0 00 1...|Concombre|
|file:/Users/samue...|2021-09-12 19:24:46| 50736|[FF D8 FF E0 00 1...|Concombre|
|file:/Users/samue...|2021-09-12 19:24:46| 50560|[FF D8 FF E0 00 1...|Concombre|
|file:/Users/samue...|2021-09-12 19:24:46| 50543|[FF D8 FF E0 00 1...|Concombre|
|file:/Users/samue...|2021-09-12 19:24:46| 50051|[FF D8 FF E0 00 1...|Concombre|
|file:/Users/samue...|2021-09-12 19:24:46| 50029|[FF D8 FF E0 00 1...|Concombre|
|file:/Users/samue...|2021-09-12 19:24:04| 35060|[FF D8 FF E0 00 1...|    Pomme|
|file:/Users/samue...|2021-09-12 19:24:04| 34794|[FF D8 FF E0 00 1...|    Pomme|
|file:/Users/samue...|2021-09-12 19:24:04| 34610|[FF D8 FF E0 00 1...|    Pomme|
|file:/Users/samue...|2021-0

In [16]:
# We can now run featurization on our entire Spark DataFrame.
# NOTE: This can take a long time (about 10 minutes) since it applies a large model to the full dataset.
features_df = df_bin.repartition(16).select(col("path"),col("classe"), featurize_udf("content").alias("features"))
#print("Nous avons" ,features_df.count(), "images de fruits.")
features_df.show()
#features_df.write.mode("overwrite").parquet("dbfs:/ml/tmp/flower_photos_features")

1/1 [==============================] - 1s 1s/step


+--------------------+---------+--------------------+
|                path|   classe|            features|
+--------------------+---------+--------------------+
|file:/Users/samue...|Concombre|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|Concombre|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|Concombre|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|    Pomme|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|    Pomme|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|  Carotte|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|  Carotte|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|  Carotte|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|    Pomme|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|Concombre|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|Concombre|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|Concombre|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|    Pomme|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|    Pomme|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|    Pomme|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|  Caro

In [17]:
features_df.first()

1/1 [==============================] - 1s 1s/step===========>     (10 + 1) / 11]


Row(path='file:/Users/samueljacquot/Desktop/P8_SAMUEL_JACQUOT/Data_low/Concombre/r0_42.jpg', classe='Concombre', features=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.31709623336792, 0.07441747188568115, 0.0, 9.940897941589355, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.5461983680725098, 0.0, 0.0, 0.0, 0.813340425491333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.228998184204102, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8105602264404297, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [18]:
df=features_df.select('path','classe','features')
df.show()

1/1 [==============================] - 1s 1s/step==>               (8 + 3) / 11]
2022-09-01 23:24:28.011875: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-01 23:24:28.011957: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
1/1 [==============================] - 1s 1s/step===========>     (10 + 1) / 11]


+--------------------+---------+--------------------+
|                path|   classe|            features|
+--------------------+---------+--------------------+
|file:/Users/samue...|Concombre|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|Concombre|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|Concombre|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|    Pomme|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|    Pomme|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|  Carotte|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|  Carotte|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|  Carotte|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|    Pomme|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|Concombre|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|Concombre|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|Concombre|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|    Pomme|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|    Pomme|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|    Pomme|[0.0, 0.0, 0.0, 0...|
|file:/Users/samue...|  Caro

# PCA

http://ethen8181.github.io/machine-learning/big_data/spark_pca.html

In [19]:
transform_vecteur_dense = udf(lambda r: Vectors.dense(r), VectorUDT())
df= df.withColumn('features_vecteurs', transform_vecteur_dense('features'))
df.show()
  

1/1 [==============================] - 2s 2s/step=======>          (9 + 2) / 11]
2022-09-01 23:24:50.026046: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
1/1 [==============================] - 1s 1s/step===========>     (10 + 1) / 11]


+--------------------+---------+--------------------+--------------------+
|                path|   classe|            features|   features_vecteurs|
+--------------------+---------+--------------------+--------------------+
|file:/Users/samue...|Concombre|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|
|file:/Users/samue...|Concombre|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|
|file:/Users/samue...|Concombre|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|
|file:/Users/samue...|    Pomme|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|
|file:/Users/samue...|    Pomme|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|
|file:/Users/samue...|  Carotte|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|
|file:/Users/samue...|  Carotte|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|
|file:/Users/samue...|  Carotte|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|
|file:/Users/samue...|    Pomme|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|
|file:/Users/samue...|Concombre|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|
|file:/Users/samue...|Con

In [20]:
scaler_std = StandardScaler(inputCol="features_vecteurs", outputCol="features_scaled", withStd=True, withMean=True)
model_std = scaler_std.fit(df)

1/1 [==============================] - 2s 2s/step                 (11 + 5) / 16]
2022-09-01 23:25:02.183952: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-01 23:25:02.183950: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-01 23:25:02.619208: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operati

In [21]:
scaler = StandardScaler(
    inputCol = 'features_vecteurs', 
    outputCol = 'features_scaled',
    withMean = True,
    withStd = True
).fit(df)

1/1 [==============================] - 2s 2s/step                 (10 + 6) / 16]
2022-09-01 23:25:12.985903: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-01 23:25:12.986038: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-01 23:25:12.985984: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operati

In [22]:
df_scaled = scaler.transform(df)
df_scaled.show(6)

22/09/01 23:25:17 WARN DAGScheduler: Broadcasting large task binary with size 1615.7 KiB


22/09/01 23:25:18 WARN DAGScheduler: Broadcasting large task binary with size 1615.7 KiB


22/09/01 23:25:19 WARN DAGScheduler: Broadcasting large task binary with size 1615.7 KiB


1/1 [==============================] - 2s 2s/step
2022-09-01 23:25:25.522174: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-01 23:25:25.522616: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


+--------------------+---------+--------------------+--------------------+--------------------+
|                path|   classe|            features|   features_vecteurs|     features_scaled|
+--------------------+---------+--------------------+--------------------+--------------------+
|file:/Users/samue...|Concombre|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|
|file:/Users/samue...|Concombre|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|
|file:/Users/samue...|Concombre|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|
|file:/Users/samue...|    Pomme|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|
|file:/Users/samue...|    Pomme|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|
|file:/Users/samue...|  Carotte|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|
+--------------------+---------+--------------------+--------------------+--------------------+
only showing top 6 rows



In [25]:
n_components = 2
pca = PCA(
    k = n_components, 
    inputCol = 'features_scaled', 
    outputCol = 'features_pca'
).fit(df_scaled)

df_pca = pca.transform(df_scaled)
df_pca.show(6)

22/09/01 23:26:36 WARN DAGScheduler: Broadcasting large task binary with size 1618.8 KiB


22/09/01 23:26:37 WARN DAGScheduler: Broadcasting large task binary with size 1618.8 KiB


22/09/01 23:26:39 WARN DAGScheduler: Broadcasting large task binary with size 1618.8 KiB


1/1 [==============================] - 2s 2s/step


22/09/01 23:26:44 WARN DAGScheduler: Broadcasting large task binary with size 1622.2 KiB


...

22/09/01 23:29:24 WARN RowMatrix: The input data was not directly cached, which may hurt performance if its parent RDDs are also uncached.
22/09/01 23:29:24 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB


22/09/01 23:29:25 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB


22/09/01 23:29:26 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB


1/1 [==============================] - 1s 1s/step


+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
|                path|   classe|            features|   features_vecteurs|     features_scaled|        features_pca|
+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
|file:/Users/samue...|Concombre|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|[91.2011077587560...|
|file:/Users/samue...|Concombre|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|[105.863803969662...|
|file:/Users/samue...|Concombre|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|[95.0217585302290...|
|file:/Users/samue...|    Pomme|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|[47.1625061753269...|
|file:/Users/samue...|    Pomme|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|[48.5876510547915...|
|file:/Users/samue...|  Carotte|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.

In [26]:
print('Explained Variance Ratio', pca.explainedVariance.toArray())

Explained Variance Ratio [0.58109525 0.41890475]


In [27]:
df_pca.select('features_pca').first()

22/09/01 23:30:31 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB


22/09/01 23:30:32 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB


22/09/01 23:30:34 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB


1/1 [==============================] - 2s 2s/step


Row(features_pca=DenseVector([91.2011, 85.2063]))

# SAVE

In [28]:
try :
    df_pca.select("path", "classe","features_pca").write.save("Finprojet.parquet")
except :
    print("Le fichier existe déjà")

Le fichier existe déjà


In [34]:
df_pandas = df_pca.toPandas()
df_pandas=df_pandas[["path","classe","features_pca"]]
print(df_pandas)

22/09/01 23:32:00 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB


1/1 [==============================] - 2s 2s/step
2022-09-01 23:32:08.088709: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


                                                 path     classe  \
0   file:/Users/samueljacquot/Desktop/P8_SAMUEL_JA...  Concombre   
1   file:/Users/samueljacquot/Desktop/P8_SAMUEL_JA...  Concombre   
2   file:/Users/samueljacquot/Desktop/P8_SAMUEL_JA...  Concombre   
3   file:/Users/samueljacquot/Desktop/P8_SAMUEL_JA...      Pomme   
4   file:/Users/samueljacquot/Desktop/P8_SAMUEL_JA...      Pomme   
5   file:/Users/samueljacquot/Desktop/P8_SAMUEL_JA...    Carotte   
6   file:/Users/samueljacquot/Desktop/P8_SAMUEL_JA...    Carotte   
7   file:/Users/samueljacquot/Desktop/P8_SAMUEL_JA...    Carotte   
8   file:/Users/samueljacquot/Desktop/P8_SAMUEL_JA...      Pomme   
9   file:/Users/samueljacquot/Desktop/P8_SAMUEL_JA...  Concombre   
10  file:/Users/samueljacquot/Desktop/P8_SAMUEL_JA...  Concombre   
11  file:/Users/samueljacquot/Desktop/P8_SAMUEL_JA...  Concombre   
12  file:/Users/samueljacquot/Desktop/P8_SAMUEL_JA...      Pomme   
13  file:/Users/samueljacquot/Desktop/P8_SAMUEL_

In [40]:
df_pandas.head()

,path,classe,features_pca
0,file:/Users/samueljacquot/Desktop/P8_SAMUEL_JA...,Concombre,"[91.20110775875607, 85.20630054449143]"
1,file:/Users/samueljacquot/Desktop/P8_SAMUEL_JA...,Concombre,"[105.8638039696628, 101.3577981652887]"
2,file:/Users/samueljacquot/Desktop/P8_SAMUEL_JA...,Concombre,"[95.02175853022906, 90.46882225418963]"
3,file:/Users/samueljacquot/Desktop/P8_SAMUEL_JA...,Pomme,"[47.16250617532698, -112.16336373670734]"
4,file:/Users/samueljacquot/Desktop/P8_SAMUEL_JA...,Pomme,"[48.58765105479157, -130.93379951973566]"


In [41]:
df_pandas.to_csv("reduction_finale.csv",index=False)

In [42]:
pd.read_csv("reduction_finale.csv").head()

,path,classe,features_pca
0,file:/Users/samueljacquot/Desktop/P8_SAMUEL_JA...,Concombre,"[91.20110775875607,85.20630054449143]"
1,file:/Users/samueljacquot/Desktop/P8_SAMUEL_JA...,Concombre,"[105.8638039696628,101.3577981652887]"
2,file:/Users/samueljacquot/Desktop/P8_SAMUEL_JA...,Concombre,"[95.02175853022906,90.46882225418963]"
3,file:/Users/samueljacquot/Desktop/P8_SAMUEL_JA...,Pomme,"[47.16250617532698,-112.16336373670734]"
4,file:/Users/samueljacquot/Desktop/P8_SAMUEL_JA...,Pomme,"[48.58765105479157,-130.93379951973566]"


In [43]:
# stop the current sparkSession
spark.stop()
print("fin")

fin


# FIN